In [ ]:
import math
import n2d2
import neurocorgi_sdk
from n2d2 import transform

In [2]:
# Change default model in N2D2 to use GPU
n2d2.global_variables.default_model = "Frame_CUDA"
n2d2.global_variables.verbosity = 0
n2d2.global_variables.cuda_device = 1

In [3]:
# Path to ImageNet dataset
ILSVRC2012_root = "/data1/is156025/DATABASE/ILSVRC2012"

# Path to ImageNet labels
ILSVRC2012_label_path = ILSVRC2012_root + "/synsets.txt"

In [5]:
# Create a dataloader
database = n2d2.database.ILSVRC2012(learn=1.0, random_partitioning=False)
database.load(ILSVRC2012_root, label_path=ILSVRC2012_label_path)

dataloader = n2d2.provider.DataProvider(database, [224, 224, 3], batch_size=1)

In [6]:
# Add transformations
transformations = transform.Composite([
    transform.ColorSpace(color_space='RGB'),
    transform.RangeAffine(first_operator='Divides', first_value=[255.0]),
    transform.RandomResizeCrop(224, 224, scale_min=0.2, scale_max=1.0, ratio_min=0.75, ratio_max=1.33, apply_to='LearnOnly'),
    transform.Rescale(256, 256, keep_aspect_ratio=True, resize_to_fit=False, apply_to='NoLearn'),
    transform.PadCrop(256, 256, apply_to='NoLearn'),
    transform.SliceExtraction(224, 224, offset_x=16, offset_y=16, apply_to='NoLearn'),
    transform.RangeAffine(first_operator='Multiplies', first_value=[255.0]),
])

dataloader.add_transformation(transformations)

In [7]:
# Import model
model = neurocorgi_sdk.models.NeuroCorgiNet([1, 3, 224, 224], weights_dir="data/imagenet_weights")

Adding cell (extractor) : bn1
Adding cell (extractor) : bn1_3x3_dw
Adding cell (extractor) : bn1_1x1
Adding cell (extractor) : bn2_3x3_dw
Adding cell (extractor) : bn2_1x1
Adding cell (extractor) : bn3_3x3_dw
Adding cell (extractor) : bn3_1x1
Adding cell (extractor) : bn4_3x3_dw
Adding cell (extractor) : bn4_1x1
Adding cell (extractor) : bn5_3x3_dw
Adding cell (extractor) : bn5_1x1
Adding cell (extractor) : bn6_3x3_dw
Adding cell (extractor) : bn6_1x1
Adding cell (extractor) : bn7_1_3x3_dw
Adding cell (extractor) : bn7_1_1x1
Adding cell (extractor) : bn7_2_3x3_dw
Adding cell (extractor) : bn7_2_1x1
Adding cell (extractor) : bn7_3_3x3_dw
Adding cell (extractor) : bn7_3_1x1
Adding cell (extractor) : bn7_4_3x3_dw
Adding cell (extractor) : bn7_4_1x1
Adding cell (extractor) : bn7_5_3x3_dw
Adding cell (extractor) : bn7_5_1x1
Adding cell (extractor) : bn8_3x3_dw
Adding cell (extractor) : bn8_1x1
Adding cell (extractor) : bn9_3x3_dw
Adding cell (extractor) : bn9_1x1
Import ../andante/weights/c

In [8]:
print(model)

'DeepNet_0' NeuroCorgiNet(
	(0): 'conv1' Conv(Frame_CUDA<float>)(nb_inputs=3, nb_outputs=32, kernel_dims=[3, 3], sub_sample_dims=[1, 1], stride_dims=[2, 2], padding_dims=[1, 1], dilation_dims=[1, 1])
	(1): 'conv1_3x3_dw' Conv(Frame_CUDA<float>)(nb_inputs=32, nb_outputs=32, kernel_dims=[3, 3], sub_sample_dims=[1, 1], stride_dims=[1, 1], padding_dims=[1, 1], dilation_dims=[1, 1])(['conv1'])
	(2): 'conv1_1x1' Conv(Frame_CUDA<float>)(nb_inputs=32, nb_outputs=64, kernel_dims=[1, 1], sub_sample_dims=[1, 1], stride_dims=[1, 1], padding_dims=[0, 0], dilation_dims=[1, 1])(['conv1_3x3_dw'])
	(3): 'conv2_3x3_dw' Conv(Frame_CUDA<float>)(nb_inputs=64, nb_outputs=64, kernel_dims=[3, 3], sub_sample_dims=[1, 1], stride_dims=[2, 2], padding_dims=[1, 1], dilation_dims=[1, 1])(['conv1_1x1'])
	(4): 'conv2_1x1' Conv(Frame_CUDA<float>)(nb_inputs=64, nb_outputs=128, kernel_dims=[1, 1], sub_sample_dims=[1, 1], stride_dims=[1, 1], padding_dims=[0, 0], dilation_dims=[1, 1])(['conv2_3x3_dw'])
	(5): 'conv3_3x3_dw

In [9]:
softmax = n2d2.cells.nn.Softmax(with_loss=True)
target = n2d2.target.Score(dataloader, top_n=1)

In [10]:
for i in range(math.ceil(database.get_nb_stimuli('Test'))):

    x = dataloader.read_batch(i)
    conv3_1x1, conv5_1x1, conv7_5_1x1, conv9_1x1, fc_out = model(x)
    pred = softmax(fc_out)
    target(pred)

    print("Test example: " + str(i) + ", test success: "
          + "{0:.3f}".format(100 * target.get_average_success()) + "%", end='\r')